# Chapter 51: AI Cost Optimization at Scale
Token tracking, caching, batching, cost modeling.


In [ ]:
import hashlib, random
from collections import OrderedDict
print("Cost Optimization ready")


In [ ]:
class Tracker:
    COST={"input":0.003,"output":0.015}
    def __init__(self): self.records=[]
    def log(self,inp,out): cost=(inp*self.COST["input"]+out*self.COST["output"])/1000; self.records.append({"in":inp,"out":out,"cost":cost})
    def summary(self): return {"calls":len(self.records),"tokens":sum(r["in"]+r["out"] for r in self.records),"cost_usd":round(sum(r["cost"] for r in self.records),4)}
random.seed(42); t=Tracker()
for _ in range(50): t.log(random.randint(200,1000),random.randint(100,500))
s=t.summary(); print("Calls:",s["calls"],"| Tokens:",s["tokens"],"| Cost: $"+str(s["cost_usd"]))


In [ ]:
class LRUCache:
    def __init__(self,cap=1000): self.cache=OrderedDict(); self.cap=cap; self.hits=self.misses=0
    def _k(self,p): return hashlib.md5(p.encode()).hexdigest()
    def get(self,p):
        k=self._k(p)
        if k in self.cache: self.cache.move_to_end(k); self.hits+=1; return self.cache[k]
        self.misses+=1; return None
    def set(self,p,r): k=self._k(p); self.cache[k]=r; self.cache.move_to_end(k)
cache=LRUCache(100)
for p in ["Explain concept "+str(i%15) for i in range(100)]:
    if cache.get(p) is None: cache.set(p,"Response for: "+p)
t=cache.hits+cache.misses
hr=cache.hits/t if t else 0
print("Cache hit rate:",round(hr*100,1),"%")
batches=[]; cur=[]; ctok=0
for r in ["Log "+str(i) for i in range(200)]:
    tok=len(r.split())*1.3
    if len(cur)>=20 or ctok+tok>8000: batches.append(cur); cur=[r]; ctok=tok
    else: cur.append(r); ctok+=tok
if cur: batches.append(cur)
print("200 requests ->",len(batches),"batches, avg size:",round(sum(len(b) for b in batches)/len(batches),1))
